In [9]:
import numpy as np
import pandas as pd
import scipy as sp
import os
import urllib
import tarfile


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [10]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = "datasets/housing"
HOUSING_URL = DOWNLOAD_ROOT + HOUSING_PATH + "/housing.tgz"
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
fetch_housing_data()

In [11]:
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [12]:
housing = load_housing_data()

In [13]:
housing.dropna(inplace=True)

In [14]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [25]:
input_fn = tf.estimator.inputs.pandas_input_fn(
    x=housing[['housing_median_age', 'total_rooms', 'total_bedrooms', 
               'population', 'households', 'median_income']],
    y=housing['median_house_value'],
    batch_size=128,
    num_epochs=2,
    shuffle=False,
    queue_capacity=1000,
    num_threads=1,
    target_column='median_house_value'
)

In [16]:
housing_median_age = tf.feature_column.numeric_column("housing_median_age")
total_rooms = tf.feature_column.numeric_column("total_rooms")
total_bedrooms = tf.feature_column.numeric_column("total_bedrooms")
population = tf.feature_column.numeric_column("population")
households = tf.feature_column.numeric_column("households")
median_income = tf.feature_column.numeric_column("median_income")

feature_columns = [
    housing_median_age, total_rooms, total_bedrooms,
    population, households, median_income
]

In [17]:
def make_model_fn(features, labels, mode, params, config):
    # Creates the input layer starting from the feature definitions of above
    input_layer = tf.feature_column.input_layer(
        features=features,
        feature_columns=feature_columns
    )

    # Get the global step
    global_step = tf.contrib.framework.get_or_create_global_step()

    model = keras.Sequential()
    # Adds a densely-connected layer with 64 units to the model:
    model.add(keras.layers.Dense(64, activation='relu'))
    # Add another:
    model.add(keras.layers.Dense(64, activation='relu'))
    # Add a output layer with 1 output units:
    model.add(keras.layers.Dense(1))


    # Now that we have our model we can compute the value of the logits 
    logits = model(input_layer)
    
    def _train_op_fn(loss):
        """Returns the op to optimize the loss."""
        
        optimizer = tf.train.AdamOptimizer()

        # Create training operation
        train_op = optimizer.minimize(
            loss=loss, global_step=tf.train.get_global_step())

        return train_op
    
    
    head = tf.contrib.estimator.regression_head(
            label_dimension=1,
            name='regression_head'
        )
    
    return head.create_estimator_spec(
            features,
            mode,
            logits,
            labels=labels,
            train_op_fn=_train_op_fn
        )

In [18]:
def metric_fn(labels, predictions):

    metrics = {}

    pred_values = predictions['predictions']

    metrics["mae"] = tf.metrics.mean_absolute_error(labels, pred_values)
    metrics["rmse"] = tf.metrics.root_mean_squared_error(labels, pred_values)

    return metrics

In [19]:
def create_estimator(model_fn, metric_fn, run_config, hparams):
    
    estimator = tf.estimator.Estimator(
        model_fn=model_fn, 
        config=run_config,
        params=hparams
    )
    
    
    estimator = tf.contrib.estimator.add_metrics(estimator, metric_fn)
    
    return estimator

In [21]:
import shutil
# shutil.rmtree('/tmp/model_dir')

In [27]:
run_config = tf.estimator.RunConfig()
run_config = run_config.replace(model_dir='/tmp/model_dir')
hparams = tf.contrib.training.HParams(
    learning_rate=.1,
)
estimator = create_estimator(model_fn=make_model_fn, metric_fn=metric_fn, run_config=run_config, hparams=hparams)

INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_master': '', '_evaluation_master': '', '_is_chief': True, '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x129f5d4a8>, '_service': None, '_model_dir': '/tmp/model_dir', '_task_type': 'worker', '_log_step_count_steps': 100, '_train_distribute': None, '_num_worker_replicas': 1, '_tf_random_seed': None, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_task_id': 0, '_num_ps_replicas': 0, '_global_id_in_cluster': 0, '_session_config': None, '_device_fn': None, '_save_checkpoints_steps': None}
INFO:tensorflow:Using config: {'_task_type': 'worker', '_master': '', '_evaluation_master': '', '_is_chief': True, '_train_distribute': None, '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x128eb7748>, '_model_dir': '/tmp/model_dir', '_service': None, '_log_step_count_steps': 100, '_num_worker_replicas

In [28]:
estimator.train(input_fn=input_fn, steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/model_dir/model.ckpt-10
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 10 into /tmp/model_dir/model.ckpt.
INFO:tensorflow:loss = 41785100000.0, step = 11
INFO:tensorflow:global_step/sec: 360.108
INFO:tensorflow:loss = 16181228000.0, step = 111 (0.279 sec)
INFO:tensorflow:global_step/sec: 356.471
INFO:tensorflow:loss = 92185030000.0, step = 211 (0.283 sec)
INFO:tensorflow:global_step/sec: 422.897
INFO:tensorflow:loss = 58363593000.0, step = 311 (0.233 sec)
INFO:tensorflow:Saving checkpoints for 330 into /tmp/model_dir/model.ckpt.
INFO:tensorflow:Loss for final step: 7228261000.0.
